LEONARDO SANTOS DE ANDRADE QUIRINO | 11.121.422-7

In [496]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error, root_mean_squared_error
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn import linear_model
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn import svm
from math import floor
import seaborn as sns
from math import ceil
import pandas as pd
import numpy as np

pokedex = pd.read_csv('pokemon.csv')
pokedex

,abilities,against_bug,against_dark,against_dragon,against_electric,against_fairy,against_fight,against_fire,against_flying,against_ghost,...,percentage_male,pokedex_number,sp_attack,sp_defense,speed,type1,type2,weight_kg,generation,is_legendary
0,"['Overgrow', 'Chlorophyll']",1.00,1.0,1.0,0.5,0.5,0.5,2.0,2.0,1.0,...,88.1,1,65,65,45,grass,poison,6.9,1,0
1,"['Overgrow', 'Chlorophyll']",1.00,1.0,1.0,0.5,0.5,0.5,2.0,2.0,1.0,...,88.1,2,80,80,60,grass,poison,13.0,1,0
2,"['Overgrow', 'Chlorophyll']",1.00,1.0,1.0,0.5,0.5,0.5,2.0,2.0,1.0,...,88.1,3,122,120,80,grass,poison,100.0,1,0
3,"['Blaze', 'Solar Power']",0.50,1.0,1.0,1.0,0.5,1.0,0.5,1.0,1.0,...,88.1,4,60,50,65,fire,NaN,8.5,1,0
4,"['Blaze', 'Solar Power']",0.50,1.0,1.0,1.0,0.5,1.0,0.5,1.0,1.0,...,88.1,5,80,65,80,fire,NaN,19.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
796,['Beast Boost'],0.25,1.0,0.5,2.0,0.5,1.0,2.0,0.5,1.0,...,NaN,797,107,101,61,steel,flying,999.9,7,1
797,['Beast Boost'],1.00,1.0,0.5,0.5,0.5,2.0,4.0,1.0,1.0,...,NaN,798,59,31,109,grass,steel,0.1,7,1
798,['Beast Boost'],2.00,0.5,2.0,0.5,4.0,2.0,0.5,1.0,0.5,...,NaN,799,97,53,43,dark,dragon,888.0,7,1
799,['Prism Armor'],2.00,2.0,1.0,1.0,1.0,0.5,1.0,1.0,2.0,...,NaN,800,127,89,79,psychic,NaN,230.0,7,1


Para realizar a regressão será utilizado o dataset pokemon.csv, um dataset com 801 Pokémon diferentes. Esse dataset apresenta diversas informações, como: atributos de batalha, altura, peso, nome japônes e ocidental, geração, etc. Todo pokémon tem um ou dois tipos, e cada tipo e combinação de tipo possui suas vantagens e desvantagens contra os demais tipos, essas informações estão contidas no dataset escolhido. O meu objetivo é treinar o algoritmo com regressão para prever qual o tipo primário e secundário com base nos valores de entrada, que serão as vantagens e desvantagens para todos os 18 tipos.

In [497]:
# essa seção é dedicada para juntas todas as vantagens e desvantagens de todos os Pokémon em uma lista só
lista = list(zip(pokedex.against_bug, pokedex.against_dark, pokedex.against_dragon, pokedex.against_electric, pokedex.against_fairy, 
                 pokedex.against_fight, pokedex.against_fire, pokedex.against_flying, pokedex.against_ghost, pokedex.against_grass, 
                 pokedex.against_ground, pokedex.against_ice, pokedex.against_normal, pokedex.against_poison, pokedex.against_psychic, 
                 pokedex.against_rock, pokedex.against_steel, pokedex.against_water))

VeD = [list(item) for item in lista]  # essa lista será a entrada da regressão
print(VeD[34])

[0.5, 0.5, 0.0, 1.0, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 2.0, 1.0]


In [498]:
# essa seção é dedicada para transformar os tipos de cada Pokémon de strings para números
tipo1 = pokedex.type1
encoder = LabelEncoder()
tipo1 = encoder.fit_transform(tipo1)

tipo2 = pokedex.type2
encoder = LabelEncoder()
tipo2 = encoder.fit_transform(tipo2)

tipagem = set(pokedex.type1)
tipagem = sorted(list(tipagem))
tipagem.append('pure')
tipagem  # a lista de tipagem é apenas uma lista com os 18 tipos. ela vai ser usada como validação para os resultados

['bug',
 'dark',
 'dragon',
 'electric',
 'fairy',
 'fighting',
 'fire',
 'flying',
 'ghost',
 'grass',
 'ground',
 'ice',
 'normal',
 'poison',
 'psychic',
 'rock',
 'steel',
 'water',
 'pure']

In [499]:
# essa seção pega os tipos, agora transformados em números, e os juntas em uma lista só
tipoum = []
tipodois = []

for i in range(0,len(tipo1)):
    tipoum.append(int(tipo1[i]))
    i += 1
    
for i in range(0,len(tipo2)):
    tipodois.append(int(tipo2[i]))
    i += 1

junta_tipos = list(zip(tipoum, tipodois))

tipos = [list(item) for item in junta_tipos]  # essa lista será a saída da regressão
tipos

[[9, 13],
 [9, 13],
 [9, 13],
 [6, 18],
 [6, 18],
 [6, 7],
 [17, 18],
 [17, 18],
 [17, 18],
 [0, 18],
 [0, 18],
 [0, 7],
 [0, 13],
 [0, 13],
 [0, 13],
 [12, 7],
 [12, 7],
 [12, 7],
 [12, 1],
 [12, 1],
 [12, 7],
 [12, 7],
 [13, 18],
 [13, 18],
 [3, 18],
 [3, 3],
 [10, 11],
 [10, 11],
 [13, 18],
 [13, 18],
 [13, 10],
 [13, 18],
 [13, 18],
 [13, 10],
 [4, 18],
 [4, 18],
 [6, 11],
 [6, 11],
 [12, 4],
 [12, 4],
 [13, 7],
 [13, 7],
 [9, 13],
 [9, 13],
 [9, 13],
 [0, 9],
 [0, 9],
 [0, 13],
 [0, 13],
 [10, 10],
 [10, 10],
 [12, 1],
 [12, 1],
 [17, 18],
 [17, 18],
 [5, 18],
 [5, 18],
 [6, 18],
 [6, 18],
 [17, 18],
 [17, 18],
 [17, 5],
 [14, 18],
 [14, 18],
 [14, 18],
 [5, 18],
 [5, 18],
 [5, 18],
 [9, 13],
 [9, 13],
 [9, 13],
 [17, 13],
 [17, 13],
 [15, 10],
 [15, 10],
 [15, 10],
 [6, 18],
 [6, 18],
 [17, 14],
 [17, 14],
 [3, 16],
 [3, 16],
 [12, 7],
 [12, 7],
 [12, 7],
 [17, 18],
 [17, 11],
 [13, 13],
 [13, 13],
 [17, 18],
 [17, 11],
 [8, 13],
 [8, 13],
 [8, 13],
 [15, 10],
 [14, 18],
 [14, 18

In [500]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# separa os dados que vão para treino e teste
# sendo 80% desses dados para treino e 20% para teste
entrada_treino, entrada_teste, saida_treino, saida_teste = train_test_split(
    VeD, tipos, test_size=0.2, random_state=0)

# mostra que a quantidade de dados foi certa para cada parametro
print(len(entrada_treino))
print(len(entrada_teste))

640
161


REGRESSÃO LINEAR

In [501]:
# o primeiro método será a regressão linear
regressao = linear_model.LinearRegression()
regressao.fit(entrada_treino, saida_treino)

LinearRegression()

In [502]:
a = regressao.coef_  # "a" é o coeficiente angular para a equação da reta
b = regressao.intercept_  # "b" é o coeficiente linear para a equação da reta

print("Coef Angular:", a, "Coef Linear:", b)

Coef Angular: [[ 0.01844542  3.15951353  0.57860104  4.25190021 -5.15360875  2.74259993
  -1.84287543  0.60103138  0.90700537  1.41108563 -0.90474744  1.09490971
   2.08291826  0.03148996  0.00719677 -4.27424211  1.6577205  -1.02708825]
 [ 0.7425521   0.13539029  5.28731811  0.37249909 -4.0799977   1.06623875
   1.1043566  -1.38961139  1.27626888 -0.38215151 -1.49571505 -3.39134764
   4.21156293  1.1601044   1.6009082  -3.51092621  0.71111226  0.61786378]] Coef Linear: [ 5.26379361 11.89755333]


In [503]:
# mostra a precisão da regressão linear
print(f"Precisão: {regressao.score(entrada_treino, saida_treino) * 100:.2f}%")

Precisão: 52.33%


In [504]:
predicao = regressao.predict(entrada_teste)

print(f'R2 = {r2_score(y_true=saida_teste, y_pred=predicao) * 100:.2f}%')
print(f'MAE = {mean_absolute_error(y_true=saida_teste, y_pred=predicao):.2f}')
print(f'MAPE = {mean_absolute_percentage_error(y_true=saida_teste, y_pred=predicao)}%')
print(f'MSE = {mean_squared_error(y_true=saida_teste, y_pred=predicao):.2f}')
print(f'RMSE = {root_mean_squared_error(y_true=saida_teste, y_pred=predicao):.2f}')

R2 = 49.56%
MAE = 3.06
MAPE = 1009883709036252.9%
MSE = 16.14
RMSE = 3.94


O valor retornado por MAPE é um número absurdamente alto e ele não será usado como métrica para a minha regressão. Esse resultado se deve ao fato de boa parte dos valores serem próximos de 0, o que faz com que o MAPE dispare devido a sua sensibilidade. Os demais valores responderam muito bem para esse e os demais modelos

Para melhor análise, todos os métodos serão testados com as vantagens e desvatagens para os 18 tipos puros e algumas entradas para dois tipos diferentes. Se os resultados para cada tipo individual retornarem valores precisos, os modelos provavelmente responderam bem para mais de um tipo 

In [505]:
# vantagens e desvantagens para o tipo inseto
print(f'Entrando com dados do tipo {tipagem[0]}')
predicao = regressao.predict([VeD[9]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo sombrio
print(f'Entrando com dados do tipo {tipagem[1]}')
predicao = regressao.predict([VeD[196]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão
print(f'Entrando com dados do tipo {tipagem[2]}')
predicao = regressao.predict([VeD[146]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo elétrico
print(f'Entrando com dados do tipo {tipagem[3]}')
predicao = regressao.predict([VeD[24]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fada
print(f'Entrando com dados do tipo {tipagem[4]}')
predicao = regressao.predict([VeD[34]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo luta
print(f'Entrando com dados do tipo {tipagem[5]}')
predicao = regressao.predict([VeD[55]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fogo
print(f'Entrando com dados do tipo {tipagem[6]}')
predicao = regressao.predict([VeD[3]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo voador
print(f'Entrando com dados do tipo {tipagem[7]}')
predicao = regressao.predict([VeD[640]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fantasma
print(f'Entrando com dados do tipo {tipagem[8]}')
predicao = regressao.predict([VeD[199]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama
print(f'Entrando com dados do tipo {tipagem[9]}')
predicao = regressao.predict([VeD[113]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo solo
print(f'Entrando com dados do tipo {tipagem[10]}')
predicao = regressao.predict([VeD[26]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo gelo
print(f'Entrando com dados do tipo {tipagem[11]}')
predicao = regressao.predict([VeD[360]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo normal
print(f'Entrando com dados do tipo {tipagem[12]}')
predicao = regressao.predict([VeD[18]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo venenoso
print(f'Entrando com dados do tipo {tipagem[13]}')
predicao = regressao.predict([VeD[22]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo psiquico
print(f'Entrando com dados do tipo {tipagem[14]}')
predicao = regressao.predict([VeD[62]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo pedra
print(f'Entrando com dados do tipo {tipagem[15]}')
predicao = regressao.predict([VeD[184]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo aço
print(f'Entrando com dados do tipo {tipagem[16]}')
predicao = regressao.predict([VeD[378]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo água
print(f'Entrando com dados do tipo {tipagem[17]}')
predicao = regressao.predict([VeD[6]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama/venenoso
print('Entrando com dados do tipo grama/venenoso')
predicao = regressao.predict([[1.0, 1.0, 1.0, 0.5, 0.5, 0.5, 2.0, 2.0, 1.0, 0.25, 1.0, 2.0, 1.0, 1.0, 2.0, 1.0, 1.0, 0.5]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão/lutador
print('Entrando com dados do tipo dragão/lutador')
predicao = regressao.predict([[0.5, 0.5, 2.0, 0.5, 4.0, 1.0, 0.5, 2.0, 1.0, 0.5, 1.0, 2.0, 1.0, 1.0, 2.0, 0.5, 1.0, 0.5]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

Entrando com dados do tipo bug
[[ 3.46509412 12.54361214]]
Este pokémon provavelmente é do tipo: electric/normal

Entrando com dados do tipo dark
[[ 6.17262973 11.35603459]]
Este pokémon provavelmente é do tipo: fire/ice

Entrando com dados do tipo dragon
[[ 5.72904026 12.89366803]]
Este pokémon provavelmente é do tipo: fighting/normal

Entrando com dados do tipo electric
[[ 6.44557585 14.5912642 ]]
Este pokémon provavelmente é do tipo: fire/psychic

Entrando com dados do tipo fairy
[[ 8.75597933 11.54578721]]
Este pokémon provavelmente é do tipo: ghost/ice

Entrando com dados do tipo fighting
[[ 6.60841033 13.38177025]]
Este pokémon provavelmente é do tipo: fire/poison

Entrando com dados do tipo fire
[[ 5.80673299 14.1929397 ]]
Este pokémon provavelmente é do tipo: fighting/psychic

Entrando com dados do tipo flying
[[10.4968991  10.18659985]]
Este pokémon provavelmente é do tipo: ground/ground

Entrando com dados do tipo ghost
[[ 9.82168236 11.11650847]]
Este pokémon provavelmente é

Mesmo com uma precisão de 52,33% e boas métricas, a regressão linear apresenta muita descrepancia para os 18 tipos e consequentemente para mais de um tipo. Muitos dos valores chegam perto e muitos apresentam uma difereça significativa

KNN

O segundo método utilizado será o KNN

In [506]:
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor(n_neighbors=2)
knn.fit(entrada_treino, saida_treino)


KNeighborsRegressor(n_neighbors=2)

In [507]:
# mostra a precisão do KNN
print(f"Precisão: {knn.score(entrada_treino, saida_treino) * 100:.2f}%")

Precisão: 81.55%


In [508]:
predicao = knn.predict(entrada_teste)

print(f'R2 = {r2_score(y_true=saida_teste, y_pred=predicao) * 100:.2f}%')
print(f'MAE = {mean_absolute_error(y_true=saida_teste, y_pred=predicao):.2f}')
print(f'MSE = {mean_squared_error(y_true=saida_teste, y_pred=predicao):.2f}')
print(f'RMSE = {root_mean_squared_error(y_true=saida_teste, y_pred=predicao):.2f}')

R2 = 57.88%
MAE = 1.66
MSE = 13.56
RMSE = 3.50


In [509]:
# vantagens e desvantagens para o tipo inseto
print(f'Entrando com dados do tipo {tipagem[0]}')
predicao = knn.predict([VeD[9]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo sombrio
print(f'Entrando com dados do tipo {tipagem[1]}')
predicao = knn.predict([VeD[196]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão
print(f'Entrando com dados do tipo {tipagem[2]}')
predicao = knn.predict([VeD[146]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo elétrico
print(f'Entrando com dados do tipo {tipagem[3]}')
predicao = knn.predict([VeD[24]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fada
print(f'Entrando com dados do tipo {tipagem[4]}')
predicao = knn.predict([VeD[34]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo luta
print(f'Entrando com dados do tipo {tipagem[5]}')
predicao = knn.predict([VeD[55]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fogo
print(f'Entrando com dados do tipo {tipagem[6]}')
predicao = knn.predict([VeD[3]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo voador
print(f'Entrando com dados do tipo {tipagem[7]}')
predicao = knn.predict([VeD[640]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fantasma
print(f'Entrando com dados do tipo {tipagem[8]}')
predicao = knn.predict([VeD[199]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama
print(f'Entrando com dados do tipo {tipagem[9]}')
predicao = knn.predict([VeD[113]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo solo
print(f'Entrando com dados do tipo {tipagem[10]}')
predicao = knn.predict([VeD[26]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo gelo
print(f'Entrando com dados do tipo {tipagem[11]}')
predicao = knn.predict([VeD[360]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo normal
print(f'Entrando com dados do tipo {tipagem[12]}')
predicao = knn.predict([VeD[18]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo venenoso
print(f'Entrando com dados do tipo {tipagem[13]}')
predicao = knn.predict([VeD[22]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo psiquico
print(f'Entrando com dados do tipo {tipagem[14]}')
predicao = knn.predict([VeD[62]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo pedra
print(f'Entrando com dados do tipo {tipagem[15]}')
predicao = knn.predict([VeD[184]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo aço
print(f'Entrando com dados do tipo {tipagem[16]}')
predicao = knn.predict([VeD[378]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo água
print(f'Entrando com dados do tipo {tipagem[17]}')
predicao = knn.predict([VeD[6]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama/venenoso
print('Entrando com dados do tipo grama/venenoso')
predicao = knn.predict([[1.0, 1.0, 1.0, 0.5, 0.5, 0.5, 2.0, 2.0, 1.0, 0.25, 1.0, 2.0, 1.0, 1.0, 2.0, 1.0, 1.0, 0.5]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão/lutador
print('Entrando com dados do tipo dragão/lutador')
predicao = knn.predict([[0.5, 0.5, 2.0, 0.5, 4.0, 1.0, 0.5, 2.0, 1.0, 0.5, 1.0, 2.0, 1.0, 1.0, 2.0, 0.5, 1.0, 0.5]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

Entrando com dados do tipo bug
[[ 0. 18.]]
Este pokémon provavelmente é do tipo: bug/pure

Entrando com dados do tipo dark
[[ 1. 18.]]
Este pokémon provavelmente é do tipo: dark/pure

Entrando com dados do tipo dragon
[[ 2. 18.]]
Este pokémon provavelmente é do tipo: dragon/pure

Entrando com dados do tipo electric
[[ 3. 18.]]
Este pokémon provavelmente é do tipo: electric/pure

Entrando com dados do tipo fairy
[[ 4. 18.]]
Este pokémon provavelmente é do tipo: fairy/pure

Entrando com dados do tipo fighting
[[ 5. 18.]]
Este pokémon provavelmente é do tipo: fighting/pure

Entrando com dados do tipo fire
[[ 6. 18.]]
Este pokémon provavelmente é do tipo: fire/pure

Entrando com dados do tipo flying
[[ 9.5 12.5]]
Este pokémon provavelmente é do tipo: grass/normal

Entrando com dados do tipo ghost
[[ 8. 18.]]
Este pokémon provavelmente é do tipo: ghost/pure

Entrando com dados do tipo grass
[[ 9. 18.]]
Este pokémon provavelmente é do tipo: grass/pure

Entrando com dados do tipo ground
[[10.

KNN respondeu muito bem para regressão, tanto para os 18 tipos quanto para dois tipos diferentes. Mesmo com métricas boas, os resultados para tipo normal e voador apresentaram descrepancias

RANDOM FOREST

O terceiro método utilizado é o Random Forest

In [510]:
rf = RandomForestRegressor()
#rf = RandomForestClassifier(n_estimators=1500, criterion='entropy', min_samples_split=10, max_depth=24, random_state=2)
rf.fit(entrada_treino, saida_treino)

RandomForestRegressor()

In [511]:
print(f'{rf.score(entrada_teste, saida_teste) * 100:.2f}%')

80.97%


In [512]:
predicao = rf.predict(entrada_teste)

print(f'R2 = {r2_score(y_true=saida_teste, y_pred=predicao) * 100:.2f}%')
print(f'MAE = {mean_absolute_error(y_true=saida_teste, y_pred=predicao):.2f}')
print(f'MSE = {mean_squared_error(y_true=saida_teste, y_pred=predicao):.2f}')
print(f'RMSE = {root_mean_squared_error(y_true=saida_teste, y_pred=predicao):.2f}')

R2 = 80.97%
MAE = 1.13
MSE = 6.07
RMSE = 2.44


In [513]:
# vantagens e desvantagens para o tipo inseto
print(f'Entrando com dados do tipo {tipagem[0]}')
predicao = rf.predict([VeD[9]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo sombrio
print(f'Entrando com dados do tipo {tipagem[1]}')
predicao = rf.predict([VeD[196]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão
print(f'Entrando com dados do tipo {tipagem[2]}')
predicao = rf.predict([VeD[146]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo elétrico
print(f'Entrando com dados do tipo {tipagem[3]}')
predicao = rf.predict([VeD[24]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fada
print(f'Entrando com dados do tipo {tipagem[4]}')
predicao = rf.predict([VeD[34]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo luta
print(f'Entrando com dados do tipo {tipagem[5]}')
predicao = rf.predict([VeD[55]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fogo
print(f'Entrando com dados do tipo {tipagem[6]}')
predicao = rf.predict([VeD[3]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo voador
print(f'Entrando com dados do tipo {tipagem[7]}')
predicao = rf.predict([VeD[640]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fantasma
print(f'Entrando com dados do tipo {tipagem[8]}')
predicao = rf.predict([VeD[199]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama
print(f'Entrando com dados do tipo {tipagem[9]}')
predicao = rf.predict([VeD[113]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo solo
print(f'Entrando com dados do tipo {tipagem[10]}')
predicao = rf.predict([VeD[26]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo gelo
print(f'Entrando com dados do tipo {tipagem[11]}')
predicao = rf.predict([VeD[360]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo normal
print(f'Entrando com dados do tipo {tipagem[12]}')
predicao = rf.predict([VeD[18]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo venenoso
print(f'Entrando com dados do tipo {tipagem[13]}')
predicao = rf.predict([VeD[22]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo psiquico
print(f'Entrando com dados do tipo {tipagem[14]}')
predicao = rf.predict([VeD[62]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo pedra
print(f'Entrando com dados do tipo {tipagem[15]}')
predicao = rf.predict([VeD[184]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo aço
print(f'Entrando com dados do tipo {tipagem[16]}')
predicao = rf.predict([VeD[378]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo água
print(f'Entrando com dados do tipo {tipagem[17]}')
predicao = rf.predict([VeD[6]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama/venenoso
print('Entrando com dados do tipo grama/venenoso')
predicao = rf.predict([[1.0, 1.0, 1.0, 0.5, 0.5, 0.5, 2.0, 2.0, 1.0, 0.25, 1.0, 2.0, 1.0, 1.0, 2.0, 1.0, 1.0, 0.5]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão/lutador
print('Entrando com dados do tipo dragão/lutador')
predicao = rf.predict([[0.5, 0.5, 2.0, 0.5, 4.0, 1.0, 0.5, 2.0, 1.0, 0.5, 1.0, 2.0, 1.0, 1.0, 2.0, 0.5, 1.0, 0.5]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

Entrando com dados do tipo bug
[[ 0. 18.]]
Este pokémon provavelmente é do tipo: bug/pure

Entrando com dados do tipo dark
[[ 1. 18.]]
Este pokémon provavelmente é do tipo: dark/pure

Entrando com dados do tipo dragon
[[ 2. 18.]]
Este pokémon provavelmente é do tipo: dragon/pure

Entrando com dados do tipo electric
[[ 3.         17.13669074]]
Este pokémon provavelmente é do tipo: electric/water

Entrando com dados do tipo fairy
[[ 4. 18.]]
Este pokémon provavelmente é do tipo: fairy/pure

Entrando com dados do tipo fighting
[[ 5. 18.]]
Este pokémon provavelmente é do tipo: fighting/pure

Entrando com dados do tipo fire
[[ 6.         17.09818708]]
Este pokémon provavelmente é do tipo: fire/water

Entrando com dados do tipo flying
[[ 9.17 13.16]]
Este pokémon provavelmente é do tipo: grass/poison

Entrando com dados do tipo ghost
[[ 8. 18.]]
Este pokémon provavelmente é do tipo: ghost/pure

Entrando com dados do tipo grass
[[ 9. 18.]]
Este pokémon provavelmente é do tipo: grass/pure

Ent

O Random Forest se comportou bem e retornou boas métricas, e mesmo que os valores retornados não tenham sido precisos na maioria dos casos, a validação por meio da lista "tipagem" mostra que 

DECISION TREE

In [514]:
from sklearn.tree import DecisionTreeRegressor

dtr = DecisionTreeRegressor()
dtr.fit(entrada_treino, saida_treino)

DecisionTreeRegressor()

In [515]:
print(f'{dtr.score(entrada_teste, saida_teste)* 100:.2f}%')

79.43%


In [516]:
predicao = dtr.predict(entrada_teste)

print(f'R2 = {r2_score(y_true=saida_teste, y_pred=predicao) * 100:.2f}%')
print(f'MAE = {mean_absolute_error(y_true=saida_teste, y_pred=predicao):.2f}')
print(f'MSE = {mean_squared_error(y_true=saida_teste, y_pred=predicao):.2f}')
print(f'RMSE = {root_mean_squared_error(y_true=saida_teste, y_pred=predicao):.2f}')

R2 = 79.43%
MAE = 0.92
MSE = 6.53
RMSE = 2.55


In [517]:
# vantagens e desvantagens para o tipo inseto
print(f'Entrando com dados do tipo {tipagem[0]}')
predicao = dtr.predict([VeD[9]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo sombrio
print(f'Entrando com dados do tipo {tipagem[1]}')
predicao = dtr.predict([VeD[196]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão
print(f'Entrando com dados do tipo {tipagem[2]}')
predicao = dtr.predict([VeD[146]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo elétrico
print(f'Entrando com dados do tipo {tipagem[3]}')
predicao = dtr.predict([VeD[24]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fada
print(f'Entrando com dados do tipo {tipagem[4]}')
predicao = dtr.predict([VeD[34]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo luta
print(f'Entrando com dados do tipo {tipagem[5]}')
predicao = dtr.predict([VeD[55]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fogo
print(f'Entrando com dados do tipo {tipagem[6]}')
predicao = dtr.predict([VeD[3]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo voador
print(f'Entrando com dados do tipo {tipagem[7]}')
predicao = dtr.predict([VeD[640]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fantasma
print(f'Entrando com dados do tipo {tipagem[8]}')
predicao = dtr.predict([VeD[199]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama
print(f'Entrando com dados do tipo {tipagem[9]}')
predicao = dtr.predict([VeD[113]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo solo
print(f'Entrando com dados do tipo {tipagem[10]}')
predicao = dtr.predict([VeD[26]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo gelo
print(f'Entrando com dados do tipo {tipagem[11]}')
predicao = dtr.predict([VeD[360]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo normal
print(f'Entrando com dados do tipo {tipagem[12]}')
predicao = dtr.predict([VeD[18]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo venenoso
print(f'Entrando com dados do tipo {tipagem[13]}')
predicao = dtr.predict([VeD[22]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo psiquico
print(f'Entrando com dados do tipo {tipagem[14]}')
predicao = dtr.predict([VeD[62]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo pedra
print(f'Entrando com dados do tipo {tipagem[15]}')
predicao = dtr.predict([VeD[184]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo aço
print(f'Entrando com dados do tipo {tipagem[16]}')
predicao = dtr.predict([VeD[378]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo água
print(f'Entrando com dados do tipo {tipagem[17]}')
predicao = dtr.predict([VeD[6]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama/venenoso
print('Entrando com dados do tipo grama/venenoso')
predicao = dtr.predict([[1.0, 1.0, 1.0, 0.5, 0.5, 0.5, 2.0, 2.0, 1.0, 0.25, 1.0, 2.0, 1.0, 1.0, 2.0, 1.0, 1.0, 0.5]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão/lutador
print('Entrando com dados do tipo dragão/lutador')
predicao = dtr.predict([[0.5, 0.5, 2.0, 0.5, 4.0, 1.0, 0.5, 2.0, 1.0, 0.5, 1.0, 2.0, 1.0, 1.0, 2.0, 0.5, 1.0, 0.5]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

Entrando com dados do tipo bug
[[ 0. 18.]]
Este pokémon provavelmente é do tipo: bug/pure

Entrando com dados do tipo dark
[[ 1. 18.]]
Este pokémon provavelmente é do tipo: dark/pure

Entrando com dados do tipo dragon
[[ 2. 18.]]
Este pokémon provavelmente é do tipo: dragon/pure

Entrando com dados do tipo electric
[[ 3.         17.21052632]]
Este pokémon provavelmente é do tipo: electric/water

Entrando com dados do tipo fairy
[[ 4. 18.]]
Este pokémon provavelmente é do tipo: fairy/pure

Entrando com dados do tipo fighting
[[ 5. 18.]]
Este pokémon provavelmente é do tipo: fighting/pure

Entrando com dados do tipo fire
[[ 6.         17.17391304]]
Este pokémon provavelmente é do tipo: fire/water

Entrando com dados do tipo flying
[[ 7. 18.]]
Este pokémon provavelmente é do tipo: flying/pure

Entrando com dados do tipo ghost
[[ 8. 18.]]
Este pokémon provavelmente é do tipo: ghost/pure

Entrando com dados do tipo grass
[[ 9. 18.]]
Este pokémon provavelmente é do tipo: grass/pure

Entrando

Apesar de ter a segunda menor precisão, a Decision Tree apresentou os melhores resultados para a regressão sem PCA ou Feature Scaling. O motivo da sua precisão pode ser visto na métrica MAE, que foi a mais baixa até o momento, e isso significa que os resultados não tem altas variações para mais ou menos, já que o resultado foi 0,92

------------- PCA -------------

Os mesmos testes feitos anteriormente, serão feitos novamente, mas com PCA

In [518]:
pca = PCA(n_components=2)
duas_dimensoes = pca.fit_transform(VeD)
duas_dimensoes

array([[-1.45577192, -0.35427967],
       [-1.45577192, -0.35427967],
       [-1.45577192, -0.35427967],
       ...,
       [-0.26567667, -2.17444328],
       [-0.10206668, -0.07391654],
       [ 0.96713856,  0.39104124]])

In [519]:
# separa os dados que vão para treino e teste
# sendo 80% desses dados para treino e 20% para teste
entrada_treinoPCA, entrada_testePCA, saida_treinoPCA, saida_testePCA = train_test_split(
    duas_dimensoes, tipos, test_size=0.2, random_state=0)

# mostra que a quantidade de dados foi certa para cada parametro
print(len(entrada_treinoPCA))
print(len(entrada_testePCA))

640
161


REGRESSÃO LINEAR COM PCA

In [520]:
# comandos para importar e realizar a regressão linear
regressaoPCA = linear_model.LinearRegression()
regressaoPCA.fit(entrada_treinoPCA, saida_treinoPCA)

LinearRegression()

In [521]:
a = regressaoPCA.coef_  # "a" é o coeficiente angular para a equação da reta
b = regressaoPCA.intercept_  # "b" é o coeficiente linear para a equação da reta
print("Coef Angular:", a, "Coef Linear:", b)

Coef Angular: [[ 1.80533014  0.5608609 ]
 [ 1.12332095 -0.01783421]] Coef Linear: [ 9.59778875 13.17186911]


In [522]:
# mostra a precisão da regressão linear
print(f"Precisão: {regressaoPCA.score(entrada_treinoPCA, saida_treinoPCA) * 100:.2f}%")

Precisão: 12.16%


In [523]:
predicao = regressaoPCA.predict(entrada_testePCA)

print(f'R2 = {r2_score(y_true=saida_testePCA, y_pred=predicao) * 100:.2f}%')
print(f'MAE = {mean_absolute_error(y_true=saida_testePCA, y_pred=predicao):.2f}')
print(f'MSE = {mean_squared_error(y_true=saida_testePCA, y_pred=predicao):.2f}')
print(f'RMSE = {root_mean_squared_error(y_true=saida_testePCA, y_pred=predicao):.2f}')

R2 = 8.70%
MAE = 4.72
MSE = 28.96
RMSE = 5.37


Só pelas métricas e pela precisão, já fica claro que a Regressão Linear terá resultados ruins

In [524]:
# vantagens e desvantagens para o tipo inseto
print(f'Entrando com dados do tipo {tipagem[0]}')
predicao = regressaoPCA.predict([duas_dimensoes[9]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo sombrio
print(f'Entrando com dados do tipo {tipagem[1]}')
predicao = regressaoPCA.predict([duas_dimensoes[196]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão
print(f'Entrando com dados do tipo {tipagem[2]}')
predicao = regressaoPCA.predict([duas_dimensoes[146]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo elétrico
print(f'Entrando com dados do tipo {tipagem[3]}')
predicao = regressaoPCA.predict([duas_dimensoes[24]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fada
print(f'Entrando com dados do tipo {tipagem[4]}')
predicao = regressaoPCA.predict([duas_dimensoes[34]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo luta
print(f'Entrando com dados do tipo {tipagem[5]}')
predicao = regressaoPCA.predict([duas_dimensoes[55]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fogo
print(f'Entrando com dados do tipo {tipagem[6]}')
predicao = regressaoPCA.predict([duas_dimensoes[3]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo voador
print(f'Entrando com dados do tipo {tipagem[7]}')
predicao = regressaoPCA.predict([duas_dimensoes[640]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fantasma
print(f'Entrando com dados do tipo {tipagem[8]}')
predicao = regressaoPCA.predict([duas_dimensoes[199]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama
print(f'Entrando com dados do tipo {tipagem[9]}')
predicao = regressaoPCA.predict([duas_dimensoes[113]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo solo
print(f'Entrando com dados do tipo {tipagem[10]}')
predicao = regressaoPCA.predict([duas_dimensoes[26]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo gelo
print(f'Entrando com dados do tipo {tipagem[11]}')
predicao = regressaoPCA.predict([duas_dimensoes[360]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo normal
print(f'Entrando com dados do tipo {tipagem[12]}')
predicao = regressaoPCA.predict([duas_dimensoes[18]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo venenoso
print(f'Entrando com dados do tipo {tipagem[13]}')
predicao = regressaoPCA.predict([duas_dimensoes[22]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo psiquico
print(f'Entrando com dados do tipo {tipagem[14]}')
predicao = regressaoPCA.predict([duas_dimensoes[62]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo pedra
print(f'Entrando com dados do tipo {tipagem[15]}')
predicao = regressaoPCA.predict([duas_dimensoes[184]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo aço
print(f'Entrando com dados do tipo {tipagem[16]}')
predicao = regressaoPCA.predict([duas_dimensoes[378]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo água
print(f'Entrando com dados do tipo {tipagem[17]}')
predicao = regressaoPCA.predict([duas_dimensoes[6]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama/venenoso
print('Entrando com dados do tipo grama/venenoso')
predicao = regressaoPCA.predict([duas_dimensoes[0]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão/lutador
print('Entrando com dados do tipo dragão/lutador')
predicao = regressaoPCA.predict([duas_dimensoes[783]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

Entrando com dados do tipo bug
[[ 7.37073518 11.63794497]]
Este pokémon provavelmente é do tipo: flying/ice

Entrando com dados do tipo dark
[[ 9.59923638 13.60143893]]
Este pokémon provavelmente é do tipo: grass/poison

Entrando com dados do tipo dragon
[[ 8.39103682 12.69722511]]
Este pokémon provavelmente é do tipo: ghost/normal

Entrando com dados do tipo electric
[[10.94265862 14.05775596]]
Este pokémon provavelmente é do tipo: ground/psychic

Entrando com dados do tipo fairy
[[ 9.26849613 12.9380703 ]]
Este pokémon provavelmente é do tipo: grass/normal

Entrando com dados do tipo fighting
[[ 9.13393721 13.00881448]]
Este pokémon provavelmente é do tipo: grass/poison

Entrando com dados do tipo fire
[[11.65682185 14.0348088 ]]
Este pokémon provavelmente é do tipo: ice/psychic

Entrando com dados do tipo flying
[[ 7.9717528  11.78174299]]
Este pokémon provavelmente é do tipo: flying/ice

Entrando com dados do tipo ghost
[[10.11015341 13.20278672]]
Este pokémon provavelmente é do ti

A descrepancia nos resultados foi extremamente alta, o que inválida o modelo de regressão linear com PCA para o objetivo desejado com esse dataset

KNN COM PCA

In [525]:
knnPCA = KNeighborsRegressor(n_neighbors=3)
knnPCA.fit(entrada_treinoPCA, saida_treinoPCA)

KNeighborsRegressor(n_neighbors=3)

In [526]:
# mostra a precisão
print(f"Precisão: {knnPCA.score(entrada_treinoPCA, saida_treinoPCA) * 100:.2f}%")
predicao = knnPCA.predict(entrada_testePCA)

Precisão: 82.26%


In [527]:
print(f'R2 = {r2_score(y_true=saida_testePCA, y_pred=predicao) * 100:.2f}%')
print(f'MAE = {mean_absolute_error(y_true=saida_testePCA, y_pred=predicao):.2f}')
print(f'MSE = {mean_squared_error(y_true=saida_testePCA, y_pred=predicao):.2f}')
print(f'RMSE = {root_mean_squared_error(y_true=saida_testePCA, y_pred=predicao):.2f}')

R2 = 60.53%
MAE = 1.70
MSE = 12.49
RMSE = 3.53


In [528]:
# vantagens e desvantagens para o tipo inseto
print(f'Entrando com dados do tipo {tipagem[0]}')
predicao = knnPCA.predict([duas_dimensoes[9]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo sombrio
print(f'Entrando com dados do tipo {tipagem[1]}')
predicao = knnPCA.predict([duas_dimensoes[196]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão
print(f'Entrando com dados do tipo {tipagem[2]}')
predicao = knnPCA.predict([duas_dimensoes[146]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo elétrico
print(f'Entrando com dados do tipo {tipagem[3]}')
predicao = knnPCA.predict([duas_dimensoes[24]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fada
print(f'Entrando com dados do tipo {tipagem[4]}')
predicao = knnPCA.predict([duas_dimensoes[34]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo luta
print(f'Entrando com dados do tipo {tipagem[5]}')
predicao = knnPCA.predict([duas_dimensoes[55]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fogo
print(f'Entrando com dados do tipo {tipagem[6]}')
predicao = knnPCA.predict([duas_dimensoes[3]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo voador
print(f'Entrando com dados do tipo {tipagem[7]}')
predicao = knnPCA.predict([duas_dimensoes[640]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fantasma
print(f'Entrando com dados do tipo {tipagem[8]}')
predicao = knnPCA.predict([duas_dimensoes[199]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama
print(f'Entrando com dados do tipo {tipagem[9]}')
predicao = knnPCA.predict([duas_dimensoes[113]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo solo
print(f'Entrando com dados do tipo {tipagem[10]}')
predicao = knnPCA.predict([duas_dimensoes[26]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo gelo
print(f'Entrando com dados do tipo {tipagem[11]}')
predicao = knnPCA.predict([duas_dimensoes[360]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo normal
print(f'Entrando com dados do tipo {tipagem[12]}')
predicao = knnPCA.predict([duas_dimensoes[18]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo venenoso
print(f'Entrando com dados do tipo {tipagem[13]}')
predicao = knnPCA.predict([duas_dimensoes[22]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo psiquico
print(f'Entrando com dados do tipo {tipagem[14]}')
predicao = knnPCA.predict([duas_dimensoes[62]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo pedra
print(f'Entrando com dados do tipo {tipagem[15]}')
predicao = knnPCA.predict([duas_dimensoes[184]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo aço
print(f'Entrando com dados do tipo {tipagem[16]}')
predicao = knnPCA.predict([duas_dimensoes[378]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo água
print(f'Entrando com dados do tipo {tipagem[17]}')
predicao = knnPCA.predict([duas_dimensoes[6]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama/venenoso
print('Entrando com dados do tipo grama/venenoso')
predicao = knnPCA.predict([duas_dimensoes[0]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão/lutador
print('Entrando com dados do tipo dragão/lutador')
predicao = knnPCA.predict([duas_dimensoes[783]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

Entrando com dados do tipo bug
[[ 0. 18.]]
Este pokémon provavelmente é do tipo: bug/pure

Entrando com dados do tipo dark
[[ 1. 18.]]
Este pokémon provavelmente é do tipo: dark/pure

Entrando com dados do tipo dragon
[[ 2. 18.]]
Este pokémon provavelmente é do tipo: dragon/pure

Entrando com dados do tipo electric
[[ 3. 18.]]
Este pokémon provavelmente é do tipo: electric/pure

Entrando com dados do tipo fairy
[[ 4. 18.]]
Este pokémon provavelmente é do tipo: fairy/pure

Entrando com dados do tipo fighting
[[ 5. 18.]]
Este pokémon provavelmente é do tipo: fighting/pure

Entrando com dados do tipo fire
[[ 6. 18.]]
Este pokémon provavelmente é do tipo: fire/pure

Entrando com dados do tipo flying
[[ 6.66666667 10.33333333]]
Este pokémon provavelmente é do tipo: fire/ground

Entrando com dados do tipo ghost
[[ 8. 18.]]
Este pokémon provavelmente é do tipo: ghost/pure

Entrando com dados do tipo grass
[[ 9. 18.]]
Este pokémon provavelmente é do tipo: grass/pure

Entrando com dados do tipo

Apesar de uma precisão e métricas melhores que o KNN padrão, o KNN com PCA não respondeu tão bem quanto o anterios. Mas os resultados obtidos ainda foram bons para a maioria dos testes, havendo uma pequena descrepancia em alguns casos

RANDOM FOREST COM PCA

In [529]:
rfPCA = RandomForestRegressor()
rfPCA.fit(entrada_treinoPCA, saida_treinoPCA)

RandomForestRegressor()

In [530]:
# mostra a precisão
print(f"Precisão: {rfPCA.score(entrada_treinoPCA, saida_treinoPCA) * 100:.2f}%")

Precisão: 90.83%


In [531]:
predicao = rfPCA.predict(entrada_testePCA)

print(f'R2 = {r2_score(y_true=saida_testePCA, y_pred=predicao) * 100:.2f}%')
print(f'MAE = {mean_absolute_error(y_true=saida_testePCA, y_pred=predicao):.2f}')
print(f'MSE = {mean_squared_error(y_true=saida_testePCA, y_pred=predicao):.2f}')
print(f'RMSE = {root_mean_squared_error(y_true=saida_testePCA, y_pred=predicao):.2f}')

R2 = 78.40%
MAE = 1.20
MSE = 6.89
RMSE = 2.60


In [532]:
# vantagens e desvantagens para o tipo inseto
print(f'Entrando com dados do tipo {tipagem[0]}')
predicao = rfPCA.predict([duas_dimensoes[9]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo sombrio
print(f'Entrando com dados do tipo {tipagem[1]}')
predicao = rfPCA.predict([duas_dimensoes[196]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão
print(f'Entrando com dados do tipo {tipagem[2]}')
predicao = rfPCA.predict([duas_dimensoes[146]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo elétrico
print(f'Entrando com dados do tipo {tipagem[3]}')
predicao = rfPCA.predict([duas_dimensoes[24]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fada
print(f'Entrando com dados do tipo {tipagem[4]}')
predicao = rfPCA.predict([duas_dimensoes[34]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo luta
print(f'Entrando com dados do tipo {tipagem[5]}')
predicao = rfPCA.predict([duas_dimensoes[55]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fogo
print(f'Entrando com dados do tipo {tipagem[6]}')
predicao = rfPCA.predict([duas_dimensoes[3]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo voador
print(f'Entrando com dados do tipo {tipagem[7]}')
predicao = rfPCA.predict([duas_dimensoes[640]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fantasma
print(f'Entrando com dados do tipo {tipagem[8]}')
predicao = rfPCA.predict([duas_dimensoes[199]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama
print(f'Entrando com dados do tipo {tipagem[9]}')
predicao = rfPCA.predict([duas_dimensoes[113]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo solo
print(f'Entrando com dados do tipo {tipagem[10]}')
predicao = rfPCA.predict([duas_dimensoes[26]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo gelo
print(f'Entrando com dados do tipo {tipagem[11]}')
predicao = rfPCA.predict([duas_dimensoes[360]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo normal
print(f'Entrando com dados do tipo {tipagem[12]}')
predicao = rfPCA.predict([duas_dimensoes[18]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo venenoso
print(f'Entrando com dados do tipo {tipagem[13]}')
predicao = rfPCA.predict([duas_dimensoes[22]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo psiquico
print(f'Entrando com dados do tipo {tipagem[14]}')
predicao = rfPCA.predict([duas_dimensoes[62]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo pedra
print(f'Entrando com dados do tipo {tipagem[15]}')
predicao = rfPCA.predict([duas_dimensoes[184]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo aço
print(f'Entrando com dados do tipo {tipagem[16]}')
predicao = rfPCA.predict([duas_dimensoes[378]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo água
print(f'Entrando com dados do tipo {tipagem[17]}')
predicao = rfPCA.predict([duas_dimensoes[6]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama/venenoso
print('Entrando com dados do tipo grama/venenoso')
predicao = rfPCA.predict([duas_dimensoes[0]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão/lutador
print('Entrando com dados do tipo dragão/lutador')
predicao = rfPCA.predict([duas_dimensoes[783]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

Entrando com dados do tipo bug
[[ 0. 18.]]
Este pokémon provavelmente é do tipo: bug/pure

Entrando com dados do tipo dark
[[ 1. 18.]]
Este pokémon provavelmente é do tipo: dark/pure

Entrando com dados do tipo dragon
[[ 2. 18.]]
Este pokémon provavelmente é do tipo: dragon/pure

Entrando com dados do tipo electric
[[ 3.         17.26667283]]
Este pokémon provavelmente é do tipo: electric/water

Entrando com dados do tipo fairy
[[ 4. 18.]]
Este pokémon provavelmente é do tipo: fairy/pure

Entrando com dados do tipo fighting
[[ 5. 18.]]
Este pokémon provavelmente é do tipo: fighting/pure

Entrando com dados do tipo fire
[[ 6.         17.08790776]]
Este pokémon provavelmente é do tipo: fire/water

Entrando com dados do tipo flying
[[ 4.98757576 14.83242424]]
Este pokémon provavelmente é do tipo: fairy/psychic

Entrando com dados do tipo ghost
[[ 8. 18.]]
Este pokémon provavelmente é do tipo: ghost/pure

Entrando com dados do tipo grass
[[ 9. 18.]]
Este pokémon provavelmente é do tipo: gr

Apesar da precisão de 90%, Random Foreste com PCA entrega alguns resultados com descrepancias. Porém, a maioria é uma pequena variação que faz com que o resultado do segundo tipo seja água (17) ao invés de puro (18)

DECISION TREE COM PCA

In [533]:
dtrPCA = DecisionTreeRegressor()
dtrPCA.fit(entrada_treinoPCA, saida_treinoPCA)

DecisionTreeRegressor()

In [534]:
# mostra a precisão
print(f"Precisão: {dtrPCA.score(entrada_treinoPCA, saida_treinoPCA) * 100:.2f}%")

Precisão: 92.06%


In [535]:
predicao = dtrPCA.predict(entrada_testePCA)

print(f'R2 = {r2_score(y_true=saida_testePCA, y_pred=predicao) * 100:.2f}%')
print(f'MAE = {mean_absolute_error(y_true=saida_testePCA, y_pred=predicao):.2f}')
print(f'MSE = {mean_squared_error(y_true=saida_testePCA, y_pred=predicao):.2f}')
print(f'RMSE = {root_mean_squared_error(y_true=saida_testePCA, y_pred=predicao):.2f}')

R2 = 76.45%
MAE = 1.04
MSE = 7.50
RMSE = 2.72


In [536]:
# vantagens e desvantagens para o tipo inseto
print(f'Entrando com dados do tipo {tipagem[0]}')
predicao = dtrPCA.predict([duas_dimensoes[9]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo sombrio
print(f'Entrando com dados do tipo {tipagem[1]}')
predicao = dtrPCA.predict([duas_dimensoes[196]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão
print(f'Entrando com dados do tipo {tipagem[2]}')
predicao = dtrPCA.predict([duas_dimensoes[146]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo elétrico
print(f'Entrando com dados do tipo {tipagem[3]}')
predicao = dtrPCA.predict([duas_dimensoes[24]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fada
print(f'Entrando com dados do tipo {tipagem[4]}')
predicao = dtrPCA.predict([duas_dimensoes[34]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo luta
print(f'Entrando com dados do tipo {tipagem[5]}')
predicao = dtrPCA.predict([duas_dimensoes[55]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fogo
print(f'Entrando com dados do tipo {tipagem[6]}')
predicao = dtrPCA.predict([duas_dimensoes[3]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo voador
print(f'Entrando com dados do tipo {tipagem[7]}')
predicao = dtrPCA.predict([duas_dimensoes[640]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fantasma
print(f'Entrando com dados do tipo {tipagem[8]}')
predicao = dtrPCA.predict([duas_dimensoes[199]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama
print(f'Entrando com dados do tipo {tipagem[9]}')
predicao = dtrPCA.predict([duas_dimensoes[113]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo solo
print(f'Entrando com dados do tipo {tipagem[10]}')
predicao = dtrPCA.predict([duas_dimensoes[26]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo gelo
print(f'Entrando com dados do tipo {tipagem[11]}')
predicao = dtrPCA.predict([duas_dimensoes[360]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo normal
print(f'Entrando com dados do tipo {tipagem[12]}')
predicao = dtrPCA.predict([duas_dimensoes[18]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo venenoso
print(f'Entrando com dados do tipo {tipagem[13]}')
predicao = dtrPCA.predict([duas_dimensoes[22]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo psiquico
print(f'Entrando com dados do tipo {tipagem[14]}')
predicao = dtrPCA.predict([duas_dimensoes[62]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo pedra
print(f'Entrando com dados do tipo {tipagem[15]}')
predicao = dtrPCA.predict([duas_dimensoes[184]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo aço
print(f'Entrando com dados do tipo {tipagem[16]}')
predicao = dtrPCA.predict([duas_dimensoes[378]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo água
print(f'Entrando com dados do tipo {tipagem[17]}')
predicao = dtrPCA.predict([duas_dimensoes[6]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama/venenoso
print('Entrando com dados do tipo grama/venenoso')
predicao = dtrPCA.predict([duas_dimensoes[0]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão/lutador
print('Entrando com dados do tipo dragão/lutador')
predicao = dtrPCA.predict([duas_dimensoes[783]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

Entrando com dados do tipo bug
[[ 0. 18.]]
Este pokémon provavelmente é do tipo: bug/pure

Entrando com dados do tipo dark
[[ 1. 18.]]
Este pokémon provavelmente é do tipo: dark/pure

Entrando com dados do tipo dragon
[[ 2. 18.]]
Este pokémon provavelmente é do tipo: dragon/pure

Entrando com dados do tipo electric
[[ 3.         17.21052632]]
Este pokémon provavelmente é do tipo: electric/water

Entrando com dados do tipo fairy
[[ 4. 18.]]
Este pokémon provavelmente é do tipo: fairy/pure

Entrando com dados do tipo fighting
[[ 5. 18.]]
Este pokémon provavelmente é do tipo: fighting/pure

Entrando com dados do tipo fire
[[ 6.         17.17391304]]
Este pokémon provavelmente é do tipo: fire/water

Entrando com dados do tipo flying
[[ 7. 18.]]
Este pokémon provavelmente é do tipo: flying/pure

Entrando com dados do tipo ghost
[[ 8. 18.]]
Este pokémon provavelmente é do tipo: ghost/pure

Entrando com dados do tipo grass
[[ 9. 18.]]
Este pokémon provavelmente é do tipo: grass/pure

Entrando

Decion Tree com PCA foi o melhor modelo para PCA, ele teve boas métricas, a maior precisão e os melhores resultados. Porém, a Decision Tree sem nada ainda se mostrou melhor em todos os aspctos

------------- FEATURE SCALING -------------

Agora os mesmos testes serão feitos para Feature scaling

In [537]:
FS = StandardScaler()
padronizado = FS.fit_transform(VeD)
padronizado

array([[ 0.00627488, -0.13044138,  0.08845721, ..., -0.35927587,
         0.0330966 , -0.92111511],
       [ 0.00627488, -0.13044138,  0.08845721, ..., -0.35927587,
         0.0330966 , -0.92111511],
       [ 0.00627488, -0.13044138,  0.08845721, ..., -0.35927587,
         0.0330966 , -0.92111511],
       ...,
       [ 1.68166846, -1.27233802,  2.92262606, ..., -0.35927587,
         0.0330966 , -0.92111511],
       [ 1.68166846,  2.1533519 ,  0.08845721, ..., -0.35927587,
         0.0330966 , -0.09628202],
       [-1.2502703 , -1.27233802, -2.74571165, ..., -1.07693167,
         0.0330966 , -0.09628202]])

In [538]:
# separa os dados que vão para treino e teste
# sendo 70% desses dados para treino e 30% para teste
entrada_treinoFS, entrada_testeFS, saida_treinoFS, saida_testeFS = train_test_split(
    padronizado, tipos, test_size=0.2, random_state=0)

# mostra que a quantidade de dados foi certa para cada parametro
print(len(entrada_treinoFS))
print(len(entrada_testeFS))

640
161


In [539]:
# comandos para importar e realizar a regressão linear
regressaoFS = linear_model.LinearRegression()
regressaoFS.fit(entrada_treinoFS, saida_treinoFS)

LinearRegression()

In [540]:
a = regressaoFS.coef_  # "a" é o coeficiente angular para a equação da reta
b = regressaoFS.intercept_  # "b" é o coeficiente linear para a equação da reta
print("Coef Angular:", a, "Coef Linear:", b)

Coef Angular: [[ 0.0110096   1.38344987  0.20415193  2.78309255 -2.68936323  1.96590374
  -1.2742021   0.3630893   0.50602544  1.11250464 -0.66802644  0.80464585
   0.55393117  0.017289    0.00356149 -2.97791932  0.82853593 -0.62260369]
 [ 0.44321054  0.05928307  1.86556214  0.24382026 -2.1291092   0.76428309
   0.76357494 -0.83947868  0.71204047 -0.30128953 -1.1043714  -2.49229116
   1.12002283  0.63693468  0.7922475  -2.44610733  0.35541701  0.37453867]] Coef Linear: [ 9.55342565 13.23629795]


In [541]:
# mostra a precisão da regressão linear
print(f"Precisão: {regressaoFS.score(entrada_treinoFS, saida_treinoFS) * 100:.2f}%")

Precisão: 52.33%


In [542]:
predicao = regressaoFS.predict(entrada_testeFS)

print(f'R2 = {r2_score(y_true=saida_testeFS, y_pred=predicao) * 100:.2f}%')
print(f'MAE = {mean_absolute_error(y_true=saida_testeFS, y_pred=predicao):.2f}')
print(f'MSE = {mean_squared_error(y_true=saida_testeFS, y_pred=predicao):.2f}')
print(f'RMSE = {root_mean_squared_error(y_true=saida_testeFS, y_pred=predicao):.2f}')

R2 = 49.56%
MAE = 3.06
MSE = 16.14
RMSE = 3.94


In [543]:
# vantagens e desvantagens para o tipo inseto
print(f'Entrando com dados do tipo {tipagem[0]}')
predicao = regressaoFS.predict([padronizado[9]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo sombrio
print(f'Entrando com dados do tipo {tipagem[1]}')
predicao = regressaoFS.predict([padronizado[196]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão
print(f'Entrando com dados do tipo {tipagem[2]}')
predicao = regressaoFS.predict([padronizado[146]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo elétrico
print(f'Entrando com dados do tipo {tipagem[3]}')
predicao = regressaoFS.predict([padronizado[24]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fada
print(f'Entrando com dados do tipo {tipagem[4]}')
predicao = regressaoFS.predict([padronizado[34]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo luta
print(f'Entrando com dados do tipo {tipagem[5]}')
predicao = regressaoFS.predict([padronizado[55]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fogo
print(f'Entrando com dados do tipo {tipagem[6]}')
predicao = regressaoFS.predict([padronizado[3]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo voador
print(f'Entrando com dados do tipo {tipagem[7]}')
predicao = regressaoFS.predict([padronizado[640]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fantasma
print(f'Entrando com dados do tipo {tipagem[8]}')
predicao = regressaoFS.predict([padronizado[199]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama
print(f'Entrando com dados do tipo {tipagem[9]}')
predicao = regressaoFS.predict([padronizado[113]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo solo
print(f'Entrando com dados do tipo {tipagem[10]}')
predicao = regressaoFS.predict([padronizado[26]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo gelo
print(f'Entrando com dados do tipo {tipagem[11]}')
predicao = regressaoFS.predict([padronizado[360]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo normal
print(f'Entrando com dados do tipo {tipagem[12]}')
predicao = regressaoFS.predict([padronizado[18]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo venenoso
print(f'Entrando com dados do tipo {tipagem[13]}')
predicao = regressaoFS.predict([padronizado[22]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo psiquico
print(f'Entrando com dados do tipo {tipagem[14]}')
predicao = regressaoFS.predict([padronizado[62]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo pedra
print(f'Entrando com dados do tipo {tipagem[15]}')
predicao = regressaoFS.predict([padronizado[184]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo aço
print(f'Entrando com dados do tipo {tipagem[16]}')
predicao = regressaoFS.predict([padronizado[378]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo água
print(f'Entrando com dados do tipo {tipagem[17]}')
predicao = regressaoFS.predict([padronizado[6]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama/venenoso
print('Entrando com dados do tipo grama/venenoso')
predicao = regressaoFS.predict([padronizado[0]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão/lutador
print('Entrando com dados do tipo dragão/lutador')
predicao = regressaoFS.predict([padronizado[783]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

Entrando com dados do tipo bug
[[ 3.46509412 12.54361214]]
Este pokémon provavelmente é do tipo: electric/normal

Entrando com dados do tipo dark
[[ 6.17262973 11.35603459]]
Este pokémon provavelmente é do tipo: fire/ice

Entrando com dados do tipo dragon
[[ 5.72904026 12.89366803]]
Este pokémon provavelmente é do tipo: fighting/normal

Entrando com dados do tipo electric
[[ 6.44557585 14.5912642 ]]
Este pokémon provavelmente é do tipo: fire/psychic

Entrando com dados do tipo fairy
[[ 8.75597933 11.54578721]]
Este pokémon provavelmente é do tipo: ghost/ice

Entrando com dados do tipo fighting
[[ 6.60841033 13.38177025]]
Este pokémon provavelmente é do tipo: fire/poison

Entrando com dados do tipo fire
[[ 5.80673299 14.1929397 ]]
Este pokémon provavelmente é do tipo: fighting/psychic

Entrando com dados do tipo flying
[[10.4968991  10.18659985]]
Este pokémon provavelmente é do tipo: ground/ground

Entrando com dados do tipo ghost
[[ 9.82168236 11.11650847]]
Este pokémon provavelmente é

Os resultados foram praticamente iguais aos resultados da Regressão Linear sem nada. Isso mostra que o modelo de Regressão Linear não é o melhor para fazer esse tipo de previsão, acredito que a quantidade elevada de dados de entrada e os dois dados de saída interfiram muito no desempenho do modelo

KNN COM FEATURE SCALING

In [544]:
knnFS = KNeighborsRegressor(n_neighbors=3)
knnFS.fit(entrada_treinoFS, saida_treinoFS)

KNeighborsRegressor(n_neighbors=3)

In [545]:
# mostra a precisão da regressão linear
print(f"Precisão: {knnFS.score(entrada_treinoFS, saida_treinoFS) * 100:.2f}%")

Precisão: 82.10%


In [546]:
predicao = knnFS.predict(entrada_testeFS)

print(f'R2 = {r2_score(y_true=saida_testeFS, y_pred=predicao) * 100:.2f}%')
print(f'MAE = {mean_absolute_error(y_true=saida_testeFS, y_pred=predicao):.2f}')
print(f'MSE = {mean_squared_error(y_true=saida_testeFS, y_pred=predicao):.2f}')
print(f'RMSE = {root_mean_squared_error(y_true=saida_testeFS, y_pred=predicao):.2f}')

R2 = 56.53%
MAE = 1.75
MSE = 13.98
RMSE = 3.57


In [547]:
# vantagens e desvantagens para o tipo inseto
print(f'Entrando com dados do tipo {tipagem[0]}')
predicao = knnFS.predict([padronizado[9]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo sombrio
print(f'Entrando com dados do tipo {tipagem[1]}')
predicao = knnFS.predict([padronizado[196]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão
print(f'Entrando com dados do tipo {tipagem[2]}')
predicao = knnFS.predict([padronizado[146]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo elétrico
print(f'Entrando com dados do tipo {tipagem[3]}')
predicao = knnFS.predict([padronizado[24]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fada
print(f'Entrando com dados do tipo {tipagem[4]}')
predicao = knnFS.predict([padronizado[34]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo luta
print(f'Entrando com dados do tipo {tipagem[5]}')
predicao = knnFS.predict([padronizado[55]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fogo
print(f'Entrando com dados do tipo {tipagem[6]}')
predicao = knnFS.predict([padronizado[3]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo voador
print(f'Entrando com dados do tipo {tipagem[7]}')
predicao = knnFS.predict([padronizado[640]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fantasma
print(f'Entrando com dados do tipo {tipagem[8]}')
predicao = knnFS.predict([padronizado[199]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama
print(f'Entrando com dados do tipo {tipagem[9]}')
predicao = knnFS.predict([padronizado[113]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo solo
print(f'Entrando com dados do tipo {tipagem[10]}')
predicao = knnFS.predict([padronizado[26]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo gelo
print(f'Entrando com dados do tipo {tipagem[11]}')
predicao = knnFS.predict([padronizado[360]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo normal
print(f'Entrando com dados do tipo {tipagem[12]}')
predicao = knnFS.predict([padronizado[18]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo venenoso
print(f'Entrando com dados do tipo {tipagem[13]}')
predicao = knnFS.predict([padronizado[22]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo psiquico
print(f'Entrando com dados do tipo {tipagem[14]}')
predicao = knnFS.predict([padronizado[62]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo pedra
print(f'Entrando com dados do tipo {tipagem[15]}')
predicao = knnFS.predict([padronizado[184]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo aço
print(f'Entrando com dados do tipo {tipagem[16]}')
predicao = knnFS.predict([padronizado[378]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo água
print(f'Entrando com dados do tipo {tipagem[17]}')
predicao = knnFS.predict([padronizado[6]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama/venenoso
print('Entrando com dados do tipo grama/venenoso')
predicao = knnFS.predict([padronizado[0]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão/lutador
print('Entrando com dados do tipo dragão/lutador')
predicao = knnFS.predict([padronizado[783]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

Entrando com dados do tipo bug
[[ 0. 18.]]
Este pokémon provavelmente é do tipo: bug/pure

Entrando com dados do tipo dark
[[ 1. 18.]]
Este pokémon provavelmente é do tipo: dark/pure

Entrando com dados do tipo dragon
[[ 2. 18.]]
Este pokémon provavelmente é do tipo: dragon/pure

Entrando com dados do tipo electric
[[ 3. 18.]]
Este pokémon provavelmente é do tipo: electric/pure

Entrando com dados do tipo fairy
[[ 4. 18.]]
Este pokémon provavelmente é do tipo: fairy/pure

Entrando com dados do tipo fighting
[[ 5. 18.]]
Este pokémon provavelmente é do tipo: fighting/pure

Entrando com dados do tipo fire
[[ 6. 18.]]
Este pokémon provavelmente é do tipo: fire/pure

Entrando com dados do tipo flying
[[10.33333333 10.66666667]]
Este pokémon provavelmente é do tipo: ground/ground

Entrando com dados do tipo ghost
[[ 8. 18.]]
Este pokémon provavelmente é do tipo: ghost/pure

Entrando com dados do tipo grass
[[ 9. 18.]]
Este pokémon provavelmente é do tipo: grass/pure

Entrando com dados do ti

RANDOM FOREST COM FEATURE SCALING

In [548]:
rfFS = RandomForestRegressor()
rfFS.fit(entrada_treinoFS, saida_treinoFS)

RandomForestRegressor()

In [549]:
# mostra a precisão da regressão linear
print(f"Precisão: {rfFS.score(entrada_treinoFS, saida_treinoFS) * 100:.2f}%")

Precisão: 91.01%


In [550]:
predicao = rfFS.predict(entrada_testeFS)

print(f'R2 = {r2_score(y_true=saida_testeFS, y_pred=predicao) * 100:.2f}%')
print(f'MAE = {mean_absolute_error(y_true=saida_testeFS, y_pred=predicao):.2f}')
print(f'MSE = {mean_squared_error(y_true=saida_testeFS, y_pred=predicao):.2f}')
print(f'RMSE = {root_mean_squared_error(y_true=saida_testeFS, y_pred=predicao):.2f}')

R2 = 81.22%
MAE = 1.13
MSE = 5.99
RMSE = 2.42


In [551]:
# vantagens e desvantagens para o tipo inseto
print(f'Entrando com dados do tipo {tipagem[0]}')
predicao = rfFS.predict([padronizado[9]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo sombrio
print(f'Entrando com dados do tipo {tipagem[1]}')
predicao = rfFS.predict([padronizado[196]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão
print(f'Entrando com dados do tipo {tipagem[2]}')
predicao = rfFS.predict([padronizado[146]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo elétrico
print(f'Entrando com dados do tipo {tipagem[3]}')
predicao = rfFS.predict([padronizado[24]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fada
print(f'Entrando com dados do tipo {tipagem[4]}')
predicao = rfFS.predict([padronizado[34]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo luta
print(f'Entrando com dados do tipo {tipagem[5]}')
predicao = rfFS.predict([padronizado[55]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fogo
print(f'Entrando com dados do tipo {tipagem[6]}')
predicao = rfFS.predict([padronizado[3]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo voador
print(f'Entrando com dados do tipo {tipagem[7]}')
predicao = rfFS.predict([padronizado[640]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fantasma
print(f'Entrando com dados do tipo {tipagem[8]}')
predicao = rfFS.predict([padronizado[199]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama
print(f'Entrando com dados do tipo {tipagem[9]}')
predicao = rfFS.predict([padronizado[113]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo solo
print(f'Entrando com dados do tipo {tipagem[10]}')
predicao = rfFS.predict([padronizado[26]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo gelo
print(f'Entrando com dados do tipo {tipagem[11]}')
predicao = rfFS.predict([padronizado[360]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo normal
print(f'Entrando com dados do tipo {tipagem[12]}')
predicao = rfFS.predict([padronizado[18]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo venenoso
print(f'Entrando com dados do tipo {tipagem[13]}')
predicao = rfFS.predict([padronizado[22]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo psiquico
print(f'Entrando com dados do tipo {tipagem[14]}')
predicao = rfFS.predict([padronizado[62]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo pedra
print(f'Entrando com dados do tipo {tipagem[15]}')
predicao = rfFS.predict([padronizado[184]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo aço
print(f'Entrando com dados do tipo {tipagem[16]}')
predicao = rfFS.predict([padronizado[378]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo água
print(f'Entrando com dados do tipo {tipagem[17]}')
predicao = rfFS.predict([padronizado[6]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama/venenoso
print('Entrando com dados do tipo grama/venenoso')
predicao = rfFS.predict([padronizado[0]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão/lutador
print('Entrando com dados do tipo dragão/lutador')
predicao = rfFS.predict([padronizado[783]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

Entrando com dados do tipo bug
[[ 0. 18.]]
Este pokémon provavelmente é do tipo: bug/pure

Entrando com dados do tipo dark
[[ 1. 18.]]
Este pokémon provavelmente é do tipo: dark/pure

Entrando com dados do tipo dragon
[[ 2. 18.]]
Este pokémon provavelmente é do tipo: dragon/pure

Entrando com dados do tipo electric
[[ 3.        17.0972183]]
Este pokémon provavelmente é do tipo: electric/water

Entrando com dados do tipo fairy
[[ 4. 18.]]
Este pokémon provavelmente é do tipo: fairy/pure

Entrando com dados do tipo fighting
[[ 5. 18.]]
Este pokémon provavelmente é do tipo: fighting/pure

Entrando com dados do tipo fire
[[ 6.        17.0722905]]
Este pokémon provavelmente é do tipo: fire/water

Entrando com dados do tipo flying
[[ 9.33 13.05]]
Este pokémon provavelmente é do tipo: grass/poison

Entrando com dados do tipo ghost
[[ 8. 18.]]
Este pokémon provavelmente é do tipo: ghost/pure

Entrando com dados do tipo grass
[[ 9. 18.]]
Este pokémon provavelmente é do tipo: grass/pure

Entrand

DECISION TREE COM FEATURE SCALING

In [552]:
dtrFS = DecisionTreeRegressor()
dtrFS.fit(entrada_treinoFS, saida_treinoFS)

DecisionTreeRegressor()

In [553]:
# mostra a precisão da regressão linear
print(f"Precisão: {dtrFS.score(entrada_treinoFS, saida_treinoFS) * 100:.2f}%")

Precisão: 92.06%


In [554]:
predicao = dtrFS.predict(entrada_testeFS)

print(f'R2 = {r2_score(y_true=saida_testeFS, y_pred=predicao) * 100:.2f}%')
print(f'MAE = {mean_absolute_error(y_true=saida_testeFS, y_pred=predicao):.2f}')
print(f'MSE = {mean_squared_error(y_true=saida_testeFS, y_pred=predicao):.2f}')
print(f'RMSE = {root_mean_squared_error(y_true=saida_testeFS, y_pred=predicao):.2f}')

R2 = 81.54%
MAE = 0.92
MSE = 5.90
RMSE = 2.39


In [555]:
# vantagens e desvantagens para o tipo inseto
print(f'Entrando com dados do tipo {tipagem[0]}')
predicao = dtrFS.predict([padronizado[9]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo sombrio
print(f'Entrando com dados do tipo {tipagem[1]}')
predicao = dtrFS.predict([padronizado[196]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão
print(f'Entrando com dados do tipo {tipagem[2]}')
predicao = dtrFS.predict([padronizado[146]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo elétrico
print(f'Entrando com dados do tipo {tipagem[3]}')
predicao = dtrFS.predict([padronizado[24]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fada
print(f'Entrando com dados do tipo {tipagem[4]}')
predicao = dtrFS.predict([padronizado[34]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo luta
print(f'Entrando com dados do tipo {tipagem[5]}')
predicao = dtrFS.predict([padronizado[55]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fogo
print(f'Entrando com dados do tipo {tipagem[6]}')
predicao = dtrFS.predict([padronizado[3]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo voador
print(f'Entrando com dados do tipo {tipagem[7]}')
predicao = dtrFS.predict([padronizado[640]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fantasma
print(f'Entrando com dados do tipo {tipagem[8]}')
predicao = dtrFS.predict([padronizado[199]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama
print(f'Entrando com dados do tipo {tipagem[9]}')
predicao = dtrFS.predict([padronizado[113]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo solo
print(f'Entrando com dados do tipo {tipagem[10]}')
predicao = dtrFS.predict([padronizado[26]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo gelo
print(f'Entrando com dados do tipo {tipagem[11]}')
predicao = dtrFS.predict([padronizado[360]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo normal
print(f'Entrando com dados do tipo {tipagem[12]}')
predicao = dtrFS.predict([padronizado[18]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo venenoso
print(f'Entrando com dados do tipo {tipagem[13]}')
predicao = dtrFS.predict([padronizado[22]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo psiquico
print(f'Entrando com dados do tipo {tipagem[14]}')
predicao = dtrFS.predict([padronizado[62]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo pedra
print(f'Entrando com dados do tipo {tipagem[15]}')
predicao = dtrFS.predict([padronizado[184]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo aço
print(f'Entrando com dados do tipo {tipagem[16]}')
predicao = dtrFS.predict([padronizado[378]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo água
print(f'Entrando com dados do tipo {tipagem[17]}')
predicao = dtrFS.predict([padronizado[6]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama/venenoso
print('Entrando com dados do tipo grama/venenoso')
predicao = dtrFS.predict([padronizado[0]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão/lutador
print('Entrando com dados do tipo dragão/lutador')
predicao = dtrFS.predict([padronizado[783]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

Entrando com dados do tipo bug
[[ 0. 18.]]
Este pokémon provavelmente é do tipo: bug/pure

Entrando com dados do tipo dark
[[ 1. 18.]]
Este pokémon provavelmente é do tipo: dark/pure

Entrando com dados do tipo dragon
[[ 2. 18.]]
Este pokémon provavelmente é do tipo: dragon/pure

Entrando com dados do tipo electric
[[ 3.         17.21052632]]
Este pokémon provavelmente é do tipo: electric/water

Entrando com dados do tipo fairy
[[ 4. 18.]]
Este pokémon provavelmente é do tipo: fairy/pure

Entrando com dados do tipo fighting
[[ 5. 18.]]
Este pokémon provavelmente é do tipo: fighting/pure

Entrando com dados do tipo fire
[[ 6.         17.17391304]]
Este pokémon provavelmente é do tipo: fire/water

Entrando com dados do tipo flying
[[ 7. 18.]]
Este pokémon provavelmente é do tipo: flying/pure

Entrando com dados do tipo ghost
[[ 8. 18.]]
Este pokémon provavelmente é do tipo: ghost/pure

Entrando com dados do tipo grass
[[ 9. 18.]]
Este pokémon provavelmente é do tipo: grass/pure

Entrando

Apesar dos metódos KNN, Random Foreste e Decision Tree terem apresentado precisões maiores e métricas um pouco melhores, os resultados não mudam muito daqueles obtidos sem Feature Scaling. Isso ocorre, pois os dados de entrada não tem uma váriação significativa entre si, então eles acabam se comportando de forma parecida, e continuam com bons resultados